In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
# to make API calls
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/dell/.secret/yelp_api.json') as f:
    login = json.load(f)

In [3]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api
#search_results = yelp_api.search_query(args)

In [5]:
#define search
LOCATION= 'Philadelphia, PA'
TERM = 'Cheesesteak'

In [6]:
JSON_FILE = "Data/results_in_progress_PA_Cheesesteak.json"
JSON_FILE

'Data/results_in_progress_PA_Cheesesteak.json'

In [7]:
def create_json_file(JSON_FILE, delete_if_exists = False):
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == True:
         if delete_if_exists == True:
                print(f"[!]{JSON_FILE} already exists. Deleting previous file...")
                os.remove(JSON_FILE)
                create_json_file(JSON_FILE, delete_if_exists = False) 
         else:
            print(f"[i]{JSON_FILE}already exists.")
            
           #if it doesnt exist
    else:
        print(f"[i]{JSON_FILE} not found. Saving empty list to new file.")
        
        folder = os.path.dirname(JSON_FILE)
        
        if len(folder)>0:
            os.makedirs(folder, exist_ok=True)
            
        with open(JSON_FILE, 'w') as f:
            json.dump([],f)
        
                
        
    

In [8]:
create_json_file(JSON_FILE, delete_if_exists = True)

with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'={n_results} previous results found.')

results = yelp_api.search_query(location = LOCATION,
                               term = TERM,
                               offset = n_results)
results.keys()

total_results = results ['total']

# how many results per page
results_per_page = len(results['businesses'])

n_pages = math.ceil((results ['total']-n_results)/results_per_page)
n_pages


[!]Data/results_in_progress_PA_Cheesesteak.json already exists. Deleting previous file...
[i]Data/results_in_progress_PA_Cheesesteak.json not found. Saving empty list to new file.
=0 previous results found.


180

In [9]:
#for loop each page
for i in tqdm_notebook(range(1, n_pages+1)):
    with open(JSON_FILE , 'r') as f:
        previous_results = json.load(f)
        
    n_results = len(previous_results)
    if(n_results +results_per_page)>1000:
        print('Exceeded 1000 api calls. Stopping.')
        break
        
    results = yelp_api.search_query(location= LOCATION,
                                   term = TERM,
                                   offset=n_results)
    
    #append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results,f)
    time.sleep(.5)

  0%|          | 0/180 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping.


In [10]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,RQAF6a0akMiot5lZZnMNNw,dalessandro-s-steaks-and-hoagies-philadelphia,Dalessandro’s Steaks & Hoagies,https://s3-media3.fl.yelpcdn.com/bphoto/yGVA9f...,False,https://www.yelp.com/biz/dalessandro-s-steaks-...,3133,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.029494, 'longitude': -75.2059714}",[delivery],$$,"{'address1': '600 Wendover St', 'address2': ''...",+12154825407,(215) 482-5407,8111.519762
1,LM54ufrINJWoTN5imV8Etw,johns-roast-pork-philadelphia,John's Roast Pork,https://s3-media3.fl.yelpcdn.com/bphoto/qPyjRa...,False,https://www.yelp.com/biz/johns-roast-pork-phil...,1737,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 39.9210273851256, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '14 E Snyder Ave', 'address2': ''...",+12154631951,(215) 463-1951,9321.694746
2,-mIlmp5l4hKlp1tvHRdvTg,sonnys-famous-steaks-philadelphia,Sonny's Famous Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/c333HA...,False,https://www.yelp.com/biz/sonnys-famous-steaks-...,1876,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.94982923010674, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '228 Market St', 'address2': '', ...",+12156295760,(215) 629-5760,6264.888631
3,jxEMFqwDJXjCxmcm5t5jVQ,cleavers-philadelphia,Cleavers,https://s3-media1.fl.yelpcdn.com/bphoto/9dfyW_...,False,https://www.yelp.com/biz/cleavers-philadelphia...,1526,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.9513931274414, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '108 S 18th St', 'address2': '', ...",+12155153828,(215) 515-3828,7227.916750
4,PP3BBaVxZLcJU54uP_wL6Q,pats-king-of-steaks-philadelphia-5,pat's king of steaks,https://s3-media4.fl.yelpcdn.com/bphoto/f-Kogk...,False,https://www.yelp.com/biz/pats-king-of-steaks-p...,4520,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 39.9332010582612, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '1237 E Passyunk Ave', 'address2'...",+12154681546,(215) 468-1546,8451.476280


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,ajGUFDANNSnqUoLvZPCcPw,maces-crossing-philadelphia,Mace's Crossing,https://s3-media4.fl.yelpcdn.com/bphoto/sCxtO4...,False,https://www.yelp.com/biz/maces-crossing-philad...,135,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 39.9559173583984, 'longitude': -7...",[delivery],$$,"{'address1': '1714 Cherry St', 'address2': '',...",+12155645203,(215) 564-5203,6716.499779
996,-3m_nXlyvdKAVNNmVirpGQ,rana-halal-cuisine-philadelphia,Rana Halal Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/6S2mYu...,False,https://www.yelp.com/biz/rana-halal-cuisine-ph...,13,"[{'alias': 'mideastern', 'title': 'Middle East...",4.0,"{'latitude': 39.9584, 'longitude': -75.19281}","[pickup, delivery]",$$,"{'address1': '3513 Lancaster Ave', 'address2':...",+12152227136,(215) 222-7136,8051.241763
997,68d_MsPklM5cwIKc_FJmgw,sharkeys-grill-and-ale-house-philadelphia-4,Sharkey's Grill And Ale House,https://s3-media2.fl.yelpcdn.com/bphoto/mfaSKM...,False,https://www.yelp.com/biz/sharkeys-grill-and-al...,14,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 40.027254102643276, 'longitude': ...",[],NaN,"{'address1': '7401 State Rd', 'address2': None...",+12156138944,(215) 613-8944,8092.622839
998,ShMMSvD_xxf3qfy70T2MrA,joes-pizza-broomall,Joe's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/QdzwY2...,False,https://www.yelp.com/biz/joes-pizza-broomall?a...,38,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 39.9807964312457, 'longitude': -7...",[delivery],$,"{'address1': '38 S Sproul Rd', 'address2': '',...",+16103569848,(610) 356-9848,20940.878731
999,jFmYoL-mN5qU1tBI0A2YXA,tony-ronis-pizza-havertown-havertown-3,Tony Roni's Pizza Havertown,https://s3-media4.fl.yelpcdn.com/bphoto/WQJRV7...,False,https://www.yelp.com/biz/tony-ronis-pizza-have...,33,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 39.9827960431925, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '800 N Eagle Rd', 'address2': 'St...",+16108535656,(610) 853-5656,17222.404244


In [11]:
final_df.duplicated(subset='id').sum()

0

In [12]:
final_df.duplicated(subset='alias').sum()

0

In [13]:
final_df.duplicated(subset='name').sum()

55

In [14]:
final_df = final_df.drop_duplicates(subset='name')

In [15]:
final_df.duplicated(subset='name').sum()

0

In [16]:
final_df.to_csv('Data/final_results_PA_Cheesesteaks.csv.gz', compression = 'gzip', index = False)